# Libraries

In [1]:
!pip -q install /kaggle/input/colombian-frenchteam-problematicinternetusage/pytorch_tabnet-4.1.0-py3-none-any.whl

In [2]:
from pytorch_tabnet.tab_model import TabNetRegressor
import torch

In [3]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.metrics import cohen_kappa_score, make_scorer, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import xgboost as xgb

In [4]:
import gc
import pandas as pd
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.offline

from colorama import Fore, Style, init
from pprint import pprint

# 🚫 Suppressing warnings 🚫
import warnings
warnings.filterwarnings('ignore')

In [5]:
SEED = 42
n_splits = 5

# Load data

In [6]:
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
len(test)

20

In [7]:
test.head(5)

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,32.6909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,3.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,27.0552,NaN,NaN,Fall,2.340,Fall,46.0,64.0,Summer,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,45.9966,NaN,NaN,Winter,2.451,Summer,31.0,45.0,Winter,0.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,Summer,1.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df = pd.read_csv('/kaggle/input/colombian-frenchteam-problematicinternetusage/Dataset_problematic_internet_usage.csv')
len(df)

3960

In [9]:
df.head(5)

,Unnamed: 0,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,...,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii,Has_actigraphy_data,Train_Test_Label
0,0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,...,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0,No,isnotintrainset
1,1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,...,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0,No,isnotintrainset
2,2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,...,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0,No,isnotintrainset
3,3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,...,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0,Yes,train
4,4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,isnotintrainset


In [10]:
sii_counts = df['sii'].value_counts(dropna=False)
sii_percentages = (sii_counts / sii_counts.sum()) * 100
sii_summary = pd.DataFrame({'Count': sii_counts,'Percentage': sii_percentages})

print(sii_summary)

     Count  Percentage
sii                   
0.0   1594   40.252525
NaN   1224   30.909091
1.0    730   18.434343
2.0    378    9.545455
3.0     34    0.858586


# Processing the data

In [11]:
featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday', 'sii']

In [12]:
df = df[featuresCols]
df = df.dropna(subset='sii')
len(df)

2736

In [13]:
df_unchanged = df.copy()

In [14]:
sii_counts = df['sii'].value_counts(dropna=False)
sii_percentages = (sii_counts / sii_counts.sum()) * 100
sii_summary = pd.DataFrame({'Count': sii_counts,'Percentage': sii_percentages})

print(sii_summary)

     Count  Percentage
sii                   
0.0   1594   58.260234
1.0    730   26.681287
2.0    378   13.815789
3.0     34    1.242690


In [15]:
def update(df):
    for c in cat_c: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df

In [16]:
cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season', 
          'FGC-Season', 'BIA-Season', 'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

df = update(df)

In [17]:
def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

In [18]:
for col in cat_c:
    mapping_df = create_mapping(col, df)
    df[col] = df[col].replace(mapping_df).astype(int)

print(f'df Shape : {df.shape}')

df Shape : (2736, 59)


In [19]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_columns] = imputer.fit_transform(df[numeric_columns])

In [20]:
df.head()

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,...,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,0.0,5.0,0.0,0.0,51.000000,0.0,16.877316,46.0,50.8,26.625259,...,0.0,2.185758,0.0,2.590608,0.0,40.977839,57.647525,0.0,3.0,2.0
1,1.0,9.0,0.0,1.0,65.159266,0.0,14.035590,48.0,46.0,22.000000,...,0.0,2.185758,1.0,2.340000,1.0,46.000000,64.000000,1.0,0.0,0.0
2,1.0,10.0,1.0,2.0,71.000000,0.0,16.648696,56.5,75.6,26.625259,...,0.0,2.185758,2.0,2.170000,1.0,38.000000,54.000000,1.0,2.0,0.0
3,2.0,9.0,0.0,2.0,71.000000,1.0,18.292347,56.0,81.6,26.625259,...,0.0,2.185758,3.0,2.451000,2.0,31.000000,45.000000,2.0,0.0,1.0
5,3.0,13.0,1.0,0.0,50.000000,1.0,22.279952,59.5,112.2,26.625259,...,0.0,2.185758,4.0,4.110000,2.0,40.000000,56.000000,3.0,0.0,1.0


In [21]:
df_unchanged.head()

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,...,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,3.0,2.0
1,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,...,NaN,NaN,Fall,2.340,Fall,46.0,64.0,Summer,0.0,0.0
2,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,...,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0,0.0
3,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,...,NaN,NaN,Winter,2.451,Summer,31.0,45.0,Winter,0.0,1.0
5,Spring,13,1,Winter,50.0,Summer,22.279952,59.5,112.2,NaN,...,NaN,NaN,Spring,4.110,Summer,40.0,56.0,Spring,0.0,1.0


# Modelling 

In [22]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.metrics import cohen_kappa_score, make_scorer, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [23]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
import lightgbm as lgb
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error

In [24]:
# Custom Kappa scorer
KAPPA_SCORER = make_scorer(
    cohen_kappa_score, 
    greater_is_better=True, 
    weights='quadratic'
)

# Custom LGBM Regressor with threshold optimization
class CustomLGBMRegressor(lgb.LGBMRegressor):
    '''
    Custom LightGBM Regressor
    
    It optimizes threshold values during fitting.
    Main goal is preventing overfit on validation data.
    '''
    def fit(self, X, y, **kwargs):
        super().fit(X, y, **kwargs)
        y_pred = super().predict(X, **kwargs)
        
        self.optimizer = minimize(
            eval_preds, 
            x0=[0.5, 1.5, 2.5], 
            args=(y, y_pred), 
            method='Nelder-Mead',
        )
        
    def predict(self, X, **kwargs):
        y_pred = super().predict(X, **kwargs)
        y_pred = threshold_Rounder(y_pred, self.optimizer.x)
        return y_pred

class CustomXGBRegressor(XGBRegressor):
    def fit(self, X, y, **kwargs):
        super().fit(X, y, **kwargs)
        y_pred = super().predict(X, **kwargs)
        self.optimizer = minimize(
            eval_preds, 
            x0=[0.5, 1.5, 2.5], 
            args=(y, y_pred), 
            method='Nelder-Mead',
        )
        
    def predict(self, X, **kwargs):
        y_pred = super().predict(X, **kwargs)
        y_pred = threshold_Rounder(y_pred, self.optimizer.x)
        return y_pred

class CustomRFRegressor(RandomForestRegressor):
    def fit(self, X, y, **kwargs):
        super().fit(X, y, **kwargs)
        y_pred = super().predict(X, **kwargs)
        self.optimizer = minimize(
            eval_preds, 
            x0=[0.5, 1.5, 2.5], 
            args=(y, y_pred), 
            method='Nelder-Mead',
        )
        
    def predict(self, X, **kwargs):
        y_pred = super().predict(X, **kwargs)
        y_pred = threshold_Rounder(y_pred, self.optimizer.x)
        return y_pred

class CustomMLPRegressor(MLPRegressor):
    def fit(self, X, y, **kwargs):
        super().fit(X, y, **kwargs)
        y_pred = super().predict(X, **kwargs)
        self.optimizer = minimize(
            eval_preds, 
            x0=[0.5, 1.5, 2.5], 
            args=(y, y_pred), 
            method='Nelder-Mead',
        )
        
    def predict(self, X, **kwargs):
        y_pred = super().predict(X, **kwargs)
        y_pred = threshold_Rounder(y_pred, self.optimizer.x)
        return y_pred
    
class CustomTabNetRegressor(TabNetRegressor):
    
    def __init__(self, optimizer_params=None, **kwargs):
        """
        Initialize the TabNetRegressor with optimizer parameters.
        optimizer_params should include 'lr', 'weight_decay', etc.
        """
        if optimizer_params is None:
            optimizer_params = dict(lr=2e-2, weight_decay=1e-5)
        
        # Pass optimizer_params to the TabNetRegressor constructor
        super().__init__(optimizer_params=optimizer_params, **kwargs)

    def fit(self, X_train, y_train, max_epochs=100, batch_size=512, virtual_batch_size=64, **kwargs):
        """
        Fit the TabNet model and optimize thresholds using minimize.
        """
        # Fit the model using the inherited fit method from TabNet
        super().fit(
            X_train, y_train,
            max_epochs=max_epochs,
            batch_size=batch_size,
            virtual_batch_size=virtual_batch_size,
            **kwargs  # Additional parameters passed here (e.g., callbacks)
        )
        
        # Predict on the training data
        y_pred = super().predict(X_train)
        
        # Optimize the threshold using a separate optimization function
        self.optimizer = minimize(
            eval_preds, 
            x0=[0.5, 1.5, 2.5],  # Initial thresholds
            args=(y_train, y_pred),  # Actual and predicted labels
            method='Nelder-Mead',
        )
        
    def predict(self, X, **kwargs):
        y_pred = super().predict(X)
        y_pred = threshold_Rounder(y_pred, self.optimizer.x)
        return y_pred

In [25]:
# Helper functions
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

def eval_preds(thresholds, y_true, y_pred):
    y_pred = threshold_Rounder(y_pred, thresholds)
    score = cohen_kappa_score(y_true, y_pred, weights='quadratic')
    return -score

In [26]:
def lgb_objective(trial):
    params = {
        'objective':         'l2',
        'verbosity':         -1,
        'n_iter':            200,
        'random_state':      SEED,
        'boosting_type':     'gbdt',
        'lambda_l1':         trial.suggest_float('lambda_l1', 1e-3, 10.0, log=True),
        'lambda_l2':         trial.suggest_float('lambda_l2', 1e-3, 10.0, log=True),
        'learning_rate':     trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
        'max_depth':         trial.suggest_int('max_depth', 4, 8),
        'num_leaves':        trial.suggest_int('num_leaves', 16, 256),
        'colsample_bytree':  trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'colsample_bynode':  trial.suggest_float('colsample_bynode', 0.4, 1.0),
        'bagging_fraction':  trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq':      trial.suggest_int('bagging_freq', 1, 7),
        'min_data_in_leaf':  trial.suggest_int('min_data_in_leaf', 5, 100),
    }
    
    X = df.drop(['sii'], axis=1)
    y = df['sii']
    cv = StratifiedKFold(5, shuffle=True, random_state=SEED)
    estimator = CustomLGBMRegressor(**params)

    val_scores = cross_val_score(
        estimator=estimator, 
        X=X, y=y, 
        cv=cv, 
        scoring=KAPPA_SCORER,
    )

    return np.mean(val_scores)

# Define objective for XGB
def xgb_objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'verbosity': 0,
        'n_estimators': 200,
        'random_state': SEED,
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
        'max_depth': trial.suggest_int('max_depth', 4, 8),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.4, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
    }

    X = df.drop(['sii'], axis=1)
    y = df['sii']
    cv = StratifiedKFold(5, shuffle=True, random_state=SEED)
    estimator = CustomXGBRegressor(**params)

    val_scores = cross_val_score(
        estimator=estimator, 
        X=X, y=y, 
        cv=cv, 
        scoring=KAPPA_SCORER,
    )

    return np.mean(val_scores)

# Define objective for Random Forest
def rf_objective(trial):
    params = {
        'n_estimators': 200,
        'random_state': SEED,
        'max_depth': trial.suggest_int('max_depth', 4, 8),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_float('max_features', 0.5, 1.0),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
    }

    X = df.drop(['sii'], axis=1)
    y = df['sii']
    cv = StratifiedKFold(5, shuffle=True, random_state=SEED)
    estimator = CustomRFRegressor(**params)

    val_scores = cross_val_score(
        estimator=estimator, 
        X=X, y=y, 
        cv=cv, 
        scoring=KAPPA_SCORER,
    )

    return np.mean(val_scores)

# Define objective for MLP
def mlp_objective(trial):
    params = {
        'hidden_layer_sizes': trial.suggest_categorical('hidden_layer_sizes', [(50,), (100,), (200,), (100, 50)]),
        'activation': trial.suggest_categorical('activation', ['relu', 'tanh', 'logistic']),
        'solver': trial.suggest_categorical('solver', ['adam', 'sgd']),
        'alpha': trial.suggest_float('alpha', 1e-5, 1e-1, log=True),
        'learning_rate': trial.suggest_categorical('learning_rate', ['constant', 'invscaling', 'adaptive']),
        'random_state': SEED,
        'max_iter': 200,
    }

    X = df.drop(['sii'], axis=1)
    y = df['sii']
    cv = StratifiedKFold(5, shuffle=True, random_state=SEED)
    estimator = CustomMLPRegressor(**params)

    val_scores = cross_val_score(
        estimator=estimator, 
        X=X, y=y, 
        cv=cv, 
        scoring=KAPPA_SCORER,
    )

    return np.mean(val_scores)

def tabnet_objective(trial):
    """
    Objective function for TabNet hyperparameter tuning using Optuna.
    """
    # Suggest hyperparameters for TabNet
    params = {
        'n_d': trial.suggest_int('n_d', 32, 128),  # Width of the decision prediction layer
        'n_a': trial.suggest_int('n_a', 32, 128),  # Width of the attention embedding for each step
        'n_steps': trial.suggest_int('n_steps', 3, 10),  # Number of steps in the architecture
        'gamma': trial.suggest_float('gamma', 1.0, 3.0),  # Coefficient for feature selection regularization
        'n_independent': trial.suggest_int('n_independent', 1, 4),  # Number of independent GLU layers
        'n_shared': trial.suggest_int('n_shared', 1, 4),  # Number of shared GLU layers
        'lambda_sparse': trial.suggest_float('lambda_sparse', 1e-5, 1e-3, log=True),  # Sparsity regularization
        'mask_type': 'entmax',  # Type of attention mask
        'scheduler_fn': torch.optim.lr_scheduler.ReduceLROnPlateau,  # Learning rate scheduler
    }

    # Cross-validation setup
    X = df.drop(['sii'], axis=1)
    y = df['sii']

    # Convert X and y to NumPy arrays
    X = X.values  # Convert to NumPy array
    y = y.values  # Convert to NumPy array

    # Reshape y to be 2D for regression (n_samples, 1)
    y = y.reshape(-1, 1)

    # Create StratifiedKFold cross-validation iterator
    cv = StratifiedKFold(5, shuffle=True, random_state=SEED)

    # Create model with suggested hyperparameters
    estimator = CustomTabNetRegressor(**params)

    # Pass optimizer parameters separately
    optimizer_params = {
        'lr': trial.suggest_float('lr', 1e-3, 1e-1, log=True),  # Learning rate
        'weight_decay': trial.suggest_float('weight_decay', 1e-6, 1e-4, log=True)  # Weight decay for regularization
    }

    # Update the model with optimizer parameters
    estimator.optimizer_params.update(optimizer_params)

    # Fit the model using cross-validation
    val_scores = cross_val_score(
        estimator=estimator, 
        X=X, 
        y=y, 
        cv=cv, 
        scoring=KAPPA_SCORER,
        fit_params={'max_epochs': 100, 'batch_size': 512, 'virtual_batch_size': 64}  # Pass fit params
    )

    # Return the mean score across all folds
    return np.mean(val_scores)

In [27]:
# Create a study for each model and optimize
def optimize_model(objective, study_name):
    study = optuna.create_study(direction='maximize', study_name=study_name)
    study.optimize(objective, n_trials=30, show_progress_bar=True)
    return study.best_trial.params


In [28]:
# Optimizing for each model
best_params_lgb = optimize_model(lgb_objective, 'LGBM_Regressor')
best_params_xgb = optimize_model(xgb_objective, 'XGB_Regressor')
best_params_rf = optimize_model(rf_objective, 'RF_Regressor')
best_params_mlp = optimize_model(mlp_objective, 'MLP_Regressor')
best_params_tabnet = optimize_model(tabnet_objective, 'TabNet_Regressor')

# Print the best parameters for each model
print("Best parameters for LGBM:", best_params_lgb)
print("Best parameters for XGB:", best_params_xgb)
print("Best parameters for RF:", best_params_rf)
print("Best parameters for MLP:", best_params_mlp)
print("Best parameters for tabnet:", best_params_tabnet)

[I 2024-11-16 18:40:39,244] A new study created in memory with name: LGBM_Regressor


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2024-11-16 18:40:42,988] Trial 0 finished with value: 0.4512518713016834 and parameters: {'lambda_l1': 0.0017708795683262664, 'lambda_l2': 8.409958835962119, 'learning_rate': 0.026912733754393535, 'max_depth': 7, 'num_leaves': 58, 'colsample_bytree': 0.7395032113013296, 'colsample_bynode': 0.40823228446191595, 'bagging_fraction': 0.6455759699655655, 'bagging_freq': 2, 'min_data_in_leaf': 43}. Best is trial 0 with value: 0.4512518713016834.
[I 2024-11-16 18:40:46,041] Trial 1 finished with value: 0.4442757318808848 and parameters: {'lambda_l1': 0.9259143297794785, 'lambda_l2': 0.018111816407843873, 'learning_rate': 0.08165800377767726, 'max_depth': 4, 'num_leaves': 124, 'colsample_bytree': 0.6462895262301576, 'colsample_bynode': 0.9267272793483008, 'bagging_fraction': 0.44424928068950004, 'bagging_freq': 2, 'min_data_in_leaf': 35}. Best is trial 0 with value: 0.4512518713016834.
[I 2024-11-16 18:40:51,205] Trial 2 finished with value: 0.4466044154180895 and parameters: {'lambda_l1': 

[I 2024-11-16 18:42:22,144] A new study created in memory with name: XGB_Regressor


[I 2024-11-16 18:42:22,139] Trial 29 finished with value: 0.44927481148647574 and parameters: {'lambda_l1': 0.22028393961205484, 'lambda_l2': 7.413664617198769, 'learning_rate': 0.025262536118081744, 'max_depth': 6, 'num_leaves': 54, 'colsample_bytree': 0.7219802299991206, 'colsample_bynode': 0.4651154037744523, 'bagging_fraction': 0.8738181690590934, 'bagging_freq': 3, 'min_data_in_leaf': 42}. Best is trial 16 with value: 0.4732348048251881.


...

[I 2024-11-16 18:55:15,240] Trial 0 finished with value: 0.0 and parameters: {'hidden_layer_sizes': (100,), 'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0051531446161194525, 'learning_rate': 'adaptive'}. Best is trial 0 with value: 0.0.
[I 2024-11-16 18:55:30,414] Trial 1 finished with value: 0.04488275742220362 and parameters: {'hidden_layer_sizes': (100,), 'activation': 'logistic', 'solver': 'sgd', 'alpha': 0.08302492916877148, 'learning_rate': 'adaptive'}. Best is trial 1 with value: 0.04488275742220362.
[I 2024-11-16 18:55:56,079] Trial 2 finished with value: 0.2996765326186525 and parameters: {'hidden_layer_sizes': (200,), 'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0061918471654451015, 'learning_rate': 'invscaling'}. Best is trial 2 with value: 0.2996765326186525.
[I 2024-11-16 18:55:58,912] Trial 3 finished with value: 0.0 and parameters: {'hidden_layer_sizes': (100,), 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.07704682850691114, 'learning_rate': 'invscaling'}

[I 2024-11-16 19:02:39,589] A new study created in memory with name: TabNet_Regressor


[I 2024-11-16 19:02:39,583] Trial 29 finished with value: 0.0 and parameters: {'hidden_layer_sizes': (50,), 'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0004613981826052596, 'learning_rate': 'constant'}. Best is trial 13 with value: 0.36594237827050985.


  0%|          | 0/30 [00:00<?, ?it/s]

epoch 0  | loss: 3.62801 |  0:00:00s
epoch 1  | loss: 1.4282  |  0:00:00s
epoch 2  | loss: 0.89731 |  0:00:00s
epoch 3  | loss: 0.7598  |  0:00:01s
epoch 4  | loss: 0.66135 |  0:00:01s
epoch 5  | loss: 0.64381 |  0:00:01s
epoch 6  | loss: 0.597   |  0:00:01s
epoch 7  | loss: 0.5841  |  0:00:02s
epoch 8  | loss: 0.54997 |  0:00:02s
epoch 9  | loss: 0.57436 |  0:00:02s
epoch 10 | loss: 0.57698 |  0:00:02s
epoch 11 | loss: 0.54729 |  0:00:03s
epoch 12 | loss: 0.5389  |  0:00:03s
epoch 13 | loss: 0.52605 |  0:00:03s
epoch 14 | loss: 0.53566 |  0:00:03s
epoch 15 | loss: 0.52982 |  0:00:04s
epoch 16 | loss: 0.51765 |  0:00:04s
epoch 17 | loss: 0.50542 |  0:00:04s
epoch 18 | loss: 0.50306 |  0:00:04s
epoch 19 | loss: 0.50905 |  0:00:05s
epoch 20 | loss: 0.49913 |  0:00:05s
epoch 21 | loss: 0.49237 |  0:00:05s
epoch 22 | loss: 0.50047 |  0:00:05s
epoch 23 | loss: 0.48527 |  0:00:05s
epoch 24 | loss: 0.49345 |  0:00:06s
epoch 25 | loss: 0.49135 |  0:00:06s
epoch 26 | loss: 0.48743 |  0:00:06s
e

In [29]:
# Print the best parameters for each model
print("Best parameters for LGBM:", best_params_lgb)
print("Best parameters for XGB:", best_params_xgb)
print("Best parameters for RF:", best_params_rf)
print("Best parameters for MLP:", best_params_mlp)
print("Best parameters for tabnet:", best_params_tabnet)

Best parameters for LGBM: {'lambda_l1': 0.01243562820696084, 'lambda_l2': 0.26823782611024716, 'learning_rate': 0.02257384781325041, 'max_depth': 5, 'num_leaves': 80, 'colsample_bytree': 0.627029204977504, 'colsample_bynode': 0.7477424264864267, 'bagging_fraction': 0.5151621754424408, 'bagging_freq': 6, 'min_data_in_leaf': 77}
Best parameters for XGB: {'lambda': 0.014521434378051432, 'alpha': 4.229150525772371, 'learning_rate': 0.022671910672493873, 'max_depth': 5, 'subsample': 0.8165175643254454, 'colsample_bytree': 0.9369513651443333, 'colsample_bylevel': 0.5488984238734824, 'min_child_weight': 2}
Best parameters for RF: {'max_depth': 8, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_features': 0.5644272489742191, 'bootstrap': True}
Best parameters for MLP: {'hidden_layer_sizes': (200,), 'activation': 'logistic', 'solver': 'adam', 'alpha': 1.1106361890724691e-05, 'learning_rate': 'constant'}
Best parameters for tabnet: {'n_d': 67, 'n_a': 99, 'n_steps': 8, 'gamma': 2.490180731767

In [30]:
params_lgb = {
    'objective': 'l2',
    'verbosity': -1,
    'n_iter': 200,
    'random_state': SEED,
    'lambda_l1': 8.906831645766042,
     'lambda_l2': 0.004591584170840488,
     'learning_rate': 0.04874811806004515,
     'max_depth': 6,
     'num_leaves': 82,
     'colsample_bytree': 0.6403876590762936,
     'colsample_bynode': 0.49845692241164663,
     'bagging_fraction': 0.55635165528929,
     'bagging_freq': 1,
     'min_data_in_leaf': 70,
}

params_xgb = {
        'objective': 'reg:squarederror',
        'verbosity': 0,
        'n_estimators': 200,
        'random_state': SEED,
        'lambda': 1e-3,
        'alpha': 1e-3,
        'learning_rate': 1e-2,
        'max_depth': 4,
        'subsample': 0.5,
        'colsample_bytree': 0.4,
        'colsample_bylevel': 0.4,
        'min_child_weight': 1,
}

params_rf= {
        'n_estimators': 200,
        'random_state': SEED,
        'max_depth': 4,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'max_features': 0.5,
        'bootstrap': True,
}

params_mlp = {
        'hidden_layer_sizes': (100, 50), 
        'activation': 'relu',
        'solver': 'adam',
        'alpha': 0.053958855282467966,
        'learning_rate': 'adaptive',
        'random_state': SEED,
        'max_iter': 200,
}

params_lgb.update(best_params_lgb)
params_xgb.update(best_params_xgb)
params_rf.update(best_params_rf)
params_mlp.update(best_params_mlp)

In [31]:
# Define optimizer params
optimizer_params_tabnet = {
    'lr': best_params_tabnet['lr'],
    'weight_decay': best_params_tabnet['weight_decay']
}

params_tabnet = {
    'n_d': best_params_tabnet['n_d'],
    'n_a': best_params_tabnet['n_a'],
    'n_steps': best_params_tabnet['n_steps'],
    'gamma': best_params_tabnet['gamma'],
    'n_independent': best_params_tabnet['n_independent'],
    'n_shared': best_params_tabnet['n_shared'],
    'lambda_sparse': best_params_tabnet['lambda_sparse'],
    'mask_type': 'entmax',  # Type of attention mask
    'optimizer_params': optimizer_params_tabnet,  # Include optimizer params
    'scheduler_fn': torch.optim.lr_scheduler.ReduceLROnPlateau,  # Learning rate scheduler
}

In [32]:
X = df.drop(['sii'], axis=1)
y = df['sii']
cv = StratifiedKFold(5, shuffle=True, random_state=SEED)

In [33]:
model_lgb = CustomLGBMRegressor(**params_lgb)
model_xgb = CustomXGBRegressor(**params_xgb)
model_rf = CustomRFRegressor(**params_rf)
model_mlp = CustomMLPRegressor(**params_mlp)

In [34]:
model_tabnet = CustomTabNetRegressor(**params_tabnet)

In [35]:
val_scores_lgb = cross_val_score(model_lgb, X, y, cv=cv, scoring=KAPPA_SCORER,)
val_scores_xgb = cross_val_score(model_xgb, X, y, cv=cv, scoring=KAPPA_SCORER,)
val_scores_rf = cross_val_score(model_rf, X, y, cv=cv, scoring=KAPPA_SCORER,)
val_scores_mlp = cross_val_score(model_mlp, X, y, cv=cv, scoring=KAPPA_SCORER,)
val_scores_tabnet = cross_val_score(model_tabnet, X.values, y.values.reshape(-1, 1), cv=cv, scoring=KAPPA_SCORER,)

kappa_scores = {
    'LGBM': np.mean(val_scores_lgb),
    'XGB': np.mean(val_scores_xgb),
    'RF': np.mean(val_scores_rf),
    'MLP': np.mean(val_scores_mlp),
    'TabNet': np.mean(val_scores_tabnet),
}

epoch 0  | loss: 2.2586  |  0:00:00s
epoch 1  | loss: 0.77447 |  0:00:00s
epoch 2  | loss: 0.61181 |  0:00:00s
epoch 3  | loss: 0.56956 |  0:00:01s
epoch 4  | loss: 0.56044 |  0:00:01s
epoch 5  | loss: 0.53137 |  0:00:01s
epoch 6  | loss: 0.51731 |  0:00:02s
epoch 7  | loss: 0.50612 |  0:00:02s
epoch 8  | loss: 0.49318 |  0:00:02s
epoch 9  | loss: 0.49027 |  0:00:03s
epoch 10 | loss: 0.49582 |  0:00:03s
epoch 11 | loss: 0.48122 |  0:00:03s
epoch 12 | loss: 0.49086 |  0:00:03s
epoch 13 | loss: 0.47351 |  0:00:04s
epoch 14 | loss: 0.46768 |  0:00:04s
epoch 15 | loss: 0.47774 |  0:00:04s
epoch 16 | loss: 0.4752  |  0:00:04s
epoch 17 | loss: 0.47196 |  0:00:05s
epoch 18 | loss: 0.474   |  0:00:05s
epoch 19 | loss: 0.48501 |  0:00:05s
epoch 20 | loss: 0.47466 |  0:00:05s
epoch 21 | loss: 0.47831 |  0:00:06s
epoch 22 | loss: 0.4754  |  0:00:06s
epoch 23 | loss: 0.46636 |  0:00:06s
epoch 24 | loss: 0.46844 |  0:00:06s
epoch 25 | loss: 0.46809 |  0:00:07s
epoch 26 | loss: 0.47463 |  0:00:07s
e

In [36]:
# Print all Kappa scores at once
for model_name, kappa_score in kappa_scores.items():
    print(f'{model_name} Kappa score: {kappa_score:.4f}')

LGBM Kappa score: 0.4732
XGB Kappa score: 0.4609
RF Kappa score: 0.4466
MLP Kappa score: 0.3659
TabNet Kappa score: 0.4213


In [37]:
model_lgb.fit(X, y)
model_xgb.fit(X, y)
model_rf.fit(X, y)
model_mlp.fit(X, y)

In [38]:
model_tabnet.fit(X.values, y.values.reshape(-1, 1))

epoch 0  | loss: 31.20129|  0:00:01s
epoch 1  | loss: 11.51207|  0:00:02s
epoch 2  | loss: 13.18169|  0:00:04s
epoch 3  | loss: 4.21    |  0:00:05s
epoch 4  | loss: 3.21909 |  0:00:07s
epoch 5  | loss: 1.40864 |  0:00:08s
epoch 6  | loss: 0.93104 |  0:00:10s
epoch 7  | loss: 0.86406 |  0:00:11s
epoch 8  | loss: 0.7851  |  0:00:13s
epoch 9  | loss: 0.75519 |  0:00:15s
epoch 10 | loss: 0.67766 |  0:00:16s
epoch 11 | loss: 0.62391 |  0:00:18s
epoch 12 | loss: 0.5655  |  0:00:19s
epoch 13 | loss: 0.57482 |  0:00:20s
epoch 14 | loss: 0.55933 |  0:00:22s
epoch 15 | loss: 0.5374  |  0:00:23s
epoch 16 | loss: 0.53791 |  0:00:25s
epoch 17 | loss: 0.54086 |  0:00:26s
epoch 18 | loss: 0.55065 |  0:00:28s
epoch 19 | loss: 0.54266 |  0:00:29s
epoch 20 | loss: 0.53545 |  0:00:31s
epoch 21 | loss: 0.55346 |  0:00:33s
epoch 22 | loss: 0.53162 |  0:00:34s
epoch 23 | loss: 0.55871 |  0:00:36s
epoch 24 | loss: 0.55529 |  0:00:37s
epoch 25 | loss: 0.53567 |  0:00:38s
epoch 26 | loss: 0.54201 |  0:00:40s
e

In [39]:
# Check individual model predictions
y_pred_lgb = model_lgb.predict(X)
y_pred_xgb = model_xgb.predict(X)
y_pred_rf = model_rf.predict(X)
y_pred_mlp = model_mlp.predict(X)

# Print the first few predictions for each model to check for variance
print("LGBM predictions:", y_pred_lgb[:10])
print("XGB predictions:", y_pred_xgb[:10])
print("RF predictions:", y_pred_rf[:10])
print("MLP predictions:", y_pred_mlp[:10])

LGBM predictions: [0 0 1 0 1 1 1 1 0 0]
XGB predictions: [1 0 0 0 1 0 1 1 1 0]
RF predictions: [1 0 0 0 1 0 0 1 1 0]
MLP predictions: [0 0 1 0 0 0 1 2 0 0]


In [40]:
y_pred_tab = model_tabnet.predict(X.values).flatten()

# Print the first few predictions for each model to check for variance
print("Tabnet predictions:", y_pred_tab[:10])

Tabnet predictions: [0 0 0 0 2 0 1 1 0 0]


In [41]:
all_preds = np.array([y_pred_lgb, y_pred_xgb, y_pred_rf, y_pred_mlp, y_pred_tab])
voted_preds = np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=0, arr=all_preds)
kappa_score = cohen_kappa_score(y, voted_preds)
print(f'Majority Voting Kappa score: {kappa_score:.4f}')

Majority Voting Kappa score: 0.4082


In [42]:
all_preds = np.array([y_pred_lgb, y_pred_xgb, y_pred_rf])
voted_preds = np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=0, arr=all_preds)
kappa_score = cohen_kappa_score(y, voted_preds)
print(f'Majority Voting Kappa score: {kappa_score:.4f}')

Majority Voting Kappa score: 0.4513


In [43]:
all_preds = np.array([y_pred_lgb, y_pred_xgb, y_pred_rf,y_pred_tab])
voted_preds = np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=0, arr=all_preds)
kappa_score = cohen_kappa_score(y, voted_preds)
print(f'Majority Voting Kappa score: {kappa_score:.4f}')

Majority Voting Kappa score: 0.4419


# Seems the best model is the LGBM so far with a predicted score of 0.4719

In [44]:
model = model_lgb

# Submission 

In [45]:
test = update(test)

for col in cat_c:
    mapping_test = create_mapping(col, test)
    test[col] = test[col].replace(mapping_test).astype(int)

print(f'Test Shape : {test.shape}')

Test Shape : (20, 59)


In [46]:
numeric_columns = test.select_dtypes(include=['float64', 'int64']).columns
test[numeric_columns] = imputer.fit_transform(test[numeric_columns])

In [47]:
Submission = test[['id']]
test = test.drop('id', axis=1)
Submission['sii'] = model.predict(test)

In [48]:
Submission

,id,sii
0,00008ff9,0
1,000fd460,0
2,00105258,1
3,00115b9f,0
4,0016bb22,0
5,001f3379,1
6,0038ba98,1
7,0068a485,0
8,0069fbed,1
9,0083e397,0


In [49]:
Submission.to_csv('submission.csv', index=False)
print(Submission['sii'].value_counts())

sii
0    13
1     7
Name: count, dtype: int64
